# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
dirname = os.path.abspath('')

In [5]:
filepath = os.path.join(dirname, 'data/song_data')
song_files = get_files(filepath)
song_files[0]

'/Users/dhrebenach/Documents/Udacity Data Engineering Nanodegree/Projects/Data Modeling with Postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [6]:
# changing pandas dataframe display options

# pd.set_option('max_colwidth', None)
# pd.reset_option('max_colwidth')
# pd.set_option('display.max_rows', 999)

In [7]:
# opening the filepath (string) and parsing the json data
with open(song_files[0]) as f:
    dfRaw = pd.read_json(f, lines=True)
    df = dfRaw.where(pd.notnull(dfRaw), None)

print(df.shape)
df.head()

(1, 10)


num_songs           artist_id artist_latitude artist_longitude  \
0          1  AR7G5I41187FB4CE6C            None             None   

   artist_location artist_name             song_id            title duration  \
0  London, England    Adam Ant  SONHOTT12A8C13493C  Something Girls  233.404   

   year  
0  1982

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
# selecting just the values of specific rows, and converting to a list format
# song_data = df['value'].loc[df['key'].isin(['artist_id','song_id', 'title', 'duration', 'year'])].values.tolist()
# song_data = df['artist_id', 'song_id'].loc[0]
song_data = df[['artist_id', 'song_id', 'title', 'duration', 'year']].values.tolist()[0]
print("song_data: ", song_data)

song_data:  ['AR7G5I41187FB4CE6C', 'SONHOTT12A8C13493C', 'Something Girls', 233.40363, 1982]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
# selecting just the values of specific rows, and converting to a list format
artist_data = df[['artist_id', 'artist_latitude', 'artist_longitude', 'artist_location', 'artist_name']].values.tolist()[0]
print("artist_data: ", artist_data)

artist_data:  ['AR7G5I41187FB4CE6C', None, None, 'London, England', 'Adam Ant']


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
filepath = os.path.join(dirname, 'data/log_data')
log_files = get_files(filepath)
log_files[0]

'/Users/dhrebenach/Documents/Udacity Data Engineering Nanodegree/Projects/Data Modeling with Postgres/data/log_data/2018/11/2018-11-11-events.json'

In [13]:
# opening the filepath (string) and parsing the json data, setting it equal to 'firstLogData'
with open(log_files[0]) as f:
    dfRaw = pd.read_json(f, lines=True)
    df = dfRaw.where(pd.notnull(dfRaw), None)

print(df.shape)
df.head()

(95, 18)


artist       auth firstName gender  itemInSession  \
0                   Frumpies  Logged In  Anabelle      F              0   
1  Kenny G with Peabo Bryson  Logged In  Anabelle      F              1   
2                Biffy Clyro  Logged In  Anabelle      F              2   
3                       None  Logged In      Lily      F              0   
4                        HIM  Logged In      Lily      F              1   

  lastName   length level                                     location method  \
0  Simpson  134.478  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD    PUT   
1  Simpson  264.751  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD    PUT   
2  Simpson  189.831  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD    PUT   
3    Burns     None  free        New York-Newark-Jersey City, NY-NJ-PA    GET   
4    Burns  212.062  free        New York-Newark-Jersey City, NY-NJ-PA    PUT   

       page registration  sessionId                            song  status  \
0  NextSong  1.54104e+12        455                      Fuck Kitty     200   
1  NextSong  1.54104e+12        455  By The Time This Night Is Over     200   
2  NextSong  1.54104e+12        455                     God & Satan     200   
3      Home  1.54062e+12        456                            None     200   
4  NextSong  1.54062e+12        456                       Beautiful     200   

              ts                                          userAgent userId  
0  1541903636796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     69  
1  1541903770796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     69  
2  1541904034796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     69  
3  1541910841796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     32  
4  1541910973796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     32

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
# loc gets rows (and/or columns) with particular labels.
# iloc gets rows (and/or columns) at integer locations.

df = df.loc[df['page'] == 'NextSong']
print(df.shape)
df.head()

(73, 18)


artist       auth firstName gender  itemInSession  \
0                   Frumpies  Logged In  Anabelle      F              0   
1  Kenny G with Peabo Bryson  Logged In  Anabelle      F              1   
2                Biffy Clyro  Logged In  Anabelle      F              2   
4                        HIM  Logged In      Lily      F              1   
5                     Matmos  Logged In    Joseph      M              0   

    lastName   length level                                     location  \
0    Simpson  134.478  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
1    Simpson  264.751  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
2    Simpson  189.831  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
4      Burns  212.062  free        New York-Newark-Jersey City, NY-NJ-PA   
5  Gutierrez  1449.12  free                                 Columbia, SC   

  method      page registration  sessionId                            song  \
0    PUT  NextSong  1.54104e+12        455                      Fuck Kitty   
1    PUT  NextSong  1.54104e+12        455  By The Time This Night Is Over   
2    PUT  NextSong  1.54104e+12        455                     God & Satan   
4    PUT  NextSong  1.54062e+12        456                       Beautiful   
5    PUT  NextSong  1.54081e+12        284                 Supreme Balloon   

   status             ts                                          userAgent  \
0     200  1541903636796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
1     200  1541903770796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
2     200  1541904034796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
4     200  1541910973796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
5     200  1541911006796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...   

  userId  
0     69  
1     69  
2     69  
4     32  
5     75

In [15]:
# Convert the ts timestamp column to datetime
# Hint: the current timestamp is in milliseconds

df.loc[:, ('ts')] = pd.to_datetime(df['ts'], unit='ms')
print(df.shape)
df.head()

(73, 18)


artist       auth firstName gender  itemInSession  \
0                   Frumpies  Logged In  Anabelle      F              0   
1  Kenny G with Peabo Bryson  Logged In  Anabelle      F              1   
2                Biffy Clyro  Logged In  Anabelle      F              2   
4                        HIM  Logged In      Lily      F              1   
5                     Matmos  Logged In    Joseph      M              0   

    lastName   length level                                     location  \
0    Simpson  134.478  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
1    Simpson  264.751  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
2    Simpson  189.831  free  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD   
4      Burns  212.062  free        New York-Newark-Jersey City, NY-NJ-PA   
5  Gutierrez  1449.12  free                                 Columbia, SC   

  method      page registration  sessionId                            song  \
0    PUT  NextSong  1.54104e+12        455                      Fuck Kitty   
1    PUT  NextSong  1.54104e+12        455  By The Time This Night Is Over   
2    PUT  NextSong  1.54104e+12        455                     God & Satan   
4    PUT  NextSong  1.54062e+12        456                       Beautiful   
5    PUT  NextSong  1.54081e+12        284                 Supreme Balloon   

   status                      ts  \
0     200 2018-11-11 02:33:56.796   
1     200 2018-11-11 02:36:10.796   
2     200 2018-11-11 02:40:34.796   
4     200 2018-11-11 04:36:13.796   
5     200 2018-11-11 04:36:46.796   

                                           userAgent userId  
0  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     69  
1  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     69  
2  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...     69  
4  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...     32  
5  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...     75

In [16]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order

# Hint: use pandas' dt attribute to access easily datetimelike properties.

# Specify labels for these columns and set to column_labels
# Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

dictionary = dict([
    ('timestamp', df['ts']),
    ('hour', df['ts'].dt.hour),
    ('day', df['ts'].dt.day),
    ('week', df['ts'].dt.week),
    ('month', df['ts'].dt.month),
    ('year', df['ts'].dt.year),
    ('weekday', df['ts'].dt.weekday),
])

In [17]:
time_df = pd.DataFrame(data=dictionary)
print(time_df.shape)
time_df.head()

(73, 7)


timestamp  hour  day  week  month  year  weekday
0 2018-11-11 02:33:56.796     2   11    45     11  2018        6
1 2018-11-11 02:36:10.796     2   11    45     11  2018        6
2 2018-11-11 02:40:34.796     2   11    45     11  2018        6
4 2018-11-11 04:36:13.796     4   11    45     11  2018        6
5 2018-11-11 04:36:46.796     4   11    45     11  2018        6

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

userId firstName   lastName gender level
0     69  Anabelle    Simpson      F  free
1     69  Anabelle    Simpson      F  free
2     69  Anabelle    Simpson      F  free
4     32      Lily      Burns      F  free
5     75    Joseph  Gutierrez      M  free

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df.iterrows():
    
    print('row.song, row.artist, row.length: ', row.song, row.artist, row.length)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    print('songid, artistid: ', songid, artistid)
    
    songplay_data = [index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]

    # insert songplay record
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

row.song, row.artist, row.length:  Fuck Kitty Frumpies 134.47791
songid, artistid:  None None
row.song, row.artist, row.length:  By The Time This Night Is Over Kenny G with Peabo Bryson 264.75057
songid, artistid:  None None
row.song, row.artist, row.length:  God & Satan Biffy Clyro 189.83138
songid, artistid:  None None
row.song, row.artist, row.length:  Beautiful HIM 212.06159
songid, artistid:  None None
row.song, row.artist, row.length:  Supreme Balloon Matmos 1449.11628
songid, artistid:  None None
row.song, row.artist, row.length:  The One Gary Allan 259.83955
songid, artistid:  None None
row.song, row.artist, row.length:  Five Roses Miracle Fortress 200.9073
songid, artistid:  None None
row.song, row.artist, row.length:  Cuentale Don Omar 261.35465
songid, artistid:  None None
row.song, row.artist, row.length:  D'Evils Jay-Z 212.27057
songid, artistid:  None None
row.song, row.artist, row.length:  Easily (Album Version) Red Hot Chili Peppers 231.33995
songid, artistid:  None Non

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.